In [ ]:
# 유방암 데이터 셋 준비하기
# 1. load_breast_cancer() 함수 호출하기
# load_breast_cancer 함수를 호출하여, Bunch 클래스의 객체를 가져온다.
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

In [ ]:
# 2. 입력 데이터 확인하기
# cancer의 data와 target을 살펴본다.
# 입력 데이터인 data의 크기를 알아본다.
print(cancer.data.shape, cancer.target.shape)
# cancer에는 569개의 sample과 30개의 Feature가 있다.
# 이 중 처음 3개의 샘플을 출력한다.
cancer.data[:3] # 실수 범위의 값이고, 양수와 음수가 섞여 있는 것을 알 수 있다.

In [ ]:
# 3. 박스 플롯으로 Feature의 사분위 관찰하기
# Feature가 30개여서 산점도로 표현할 수 없다.
# Boxplot을 이용하여 각 특성의 사분위(Quartile)값을 나타낸다.
# Boxplot은 1사분위와 3사분위 값으로 상자를 그린 뒤, 2사분위(중간값) 값을 표시한다.
# 그 당므, 1사분위와 3사분위 사이 거리(interquartile range)의 1.5배만큼 위아래 거리에서 각각 가장 큰 값과 가장 작은 값까지 수염을 그린다.
import numpy as np
import matplotlib.pyplot as plt

plt.boxplot(cancer.data)
plt.xlabel("Feature")
plt.ylabel("Value")
plt.show()

In [ ]:
# 4. 눈에 띄는 Feature 살펴보기
'''
Boxplot을 보면 4, 14, 24번째 특성이 다른 특성보다 값의 분포가 훨씬 크다는 것을 알 수 있다.
다른 특성과 차이가 나는 특들을 확인한다.
- 4, 14, 24 번째 특성의 인덱스를 리스트로 묶어 전달하면, 각 인덱스의 특성을 확인할 수 있다.
'''
print(cancer.feature_names[[3, 13, 23]]) # ['mean area' 'area error' 'worst area']

In [ ]:
# 5. 타깃 데이터 확인하기
'''
해결해야할 문제는 음성 샘플과 양성 샘플을 구분하는 이진 문제이다.
- cancer.target 배열 안에는 0과 1만 들어있다.
    - 0은 음성 클래스를 의미한다.
    - 1은 양성 클래스를 의미한다.
- 넘파이의 unique() 함수를 사용하면 고유한 값을 찾아 반환한다.
    - 이때 return_counts 매개변수를 True로 지정하면 고유한 값이 등장하는 횟수까지 세어서 반환한다.
'''
np.unique(cancer.target, return_counts= True)
'''
unique() 함수가 반환한 값을 확인해보면, 두 값을 반환하고 있다.
왼쪽의 값은 cancer.target에 들어 있는 고유한 값(0, 1)을 의미한다.
- 즉, target에는 0과 1만 들어있따.
오른쪽의 값은 타깃 데이터의 고유한 값의 개수를 샌 후, 반환한 것이다.
- 즉, 위의 타깃 데이터에는 212개의 음성 클래스와 357개의 양성 클래스가 들어 있다.
'''

In [ ]:
# 6. 훈련 데이터 세트 저장하기
# 이제 예제 데이터 세트를 x, y변수에 저장한다.
x = cancer.data
y = cancer.target

In [ ]:
# 1. train_test_split() 함수를 이용하여 훈련 데이터 세트로 나누기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, stratify= y, test_size= 0.2, random_state= 42)
'''
stratify = y
- 훈련 데이터를 나눌 클래스 비율을 동일하게 만든다.
- train_test_split()함수는 기본적으로 데이터를 나누기 전에 섞지만, 일부 클래스 비율이 불균형한 경우, stratify를 y로 지정해야 한다.
'''

In [ ]:
# 2. 결과 확인하기
'''
shape 속성을 이용하여 훈련 세트와 테스트 세트의 비율을 확인한다.
'''
print(X_train.shape, X_test.shape) # 455 : 144 즉, 4:1 비율로 잘 나눠졌다.

In [ ]:
# 3. unique() 함수로 훈련세트의 타깃 확인하기
np.unique(y_train, return_counts= True) # 클래스 비율이 그대로 유지되는 것을 확인할 수 있다.

In [ ]:
'''
로지스틱 회귀 구현하기
- 로지스틱 회귀는 정방향으로 데이터가 흘러가는 과정(정방향 계산)과 가중치를 업데이트하기 위한 역방향으로 데이터가 흘러가는 과정(역방향 계산)을 구현해야 한다.
- 정방향 계산부터 역방향 계산까지 순서대로 구현한다.
'''
class LogisticNeuron:
    def __init__(self):
        self.w = None
        self.b = None
    def forpass(self, x):
        z = np.sum(x * self.w) + self.b # 직선 방정식을 계산한다.
        return z
    
    def backprop(self, x, err):
        w_grad = x * err
        b_grad = 1 * err                # 가중치에 대한 그레이디언트를 계산한다.
        return w_grad, b_grad           # 절편에 대한 그레이디언트를 계산한다.
    
    def fit(self, x, y, epochs = 100):
        self.w = np.ones(x.shape[1])                            # 가중치를 초기화한다.
        self.b = 0                                              # 절편을 초기화한다.
        for i in range(epochs):                                 # epochs만큼 반복한다.
            for x_i, y_i in zip(x, y):                          # 모든 샘플에 대해 반복한다.
                z = self.forpass(x_i)                           # 정방향 계산
                a = self.activation(z)                          # 활성화 함수 적용
                err = -(y_i - a)                                # 오차 계산
                w_grad, b_grad = self.backprop(x_i, err)        # 역방향 계산
                self.w -= w_grad                                # 가중치 업데이트
                self.b -= b_grad                                # 절편 업데이트
    
    def activation(self, z):
        a = 1 / (1 + np.exp(-z)) # 시그모이드 계산
        return a
    
    def predict(self, x):
        z = [self.forpass(x_i) for x_i in x]        # 선형 함수 적용
        a = self.activation(np.array(z))            # 활성화 함수 적용
        return a > 0.5                              # 계단 함수 적용

In [ ]:
'''
훈련하는 메서드 구현하기
- 훈련을 수행하는 fit() 메서드를 구현한다.
'''

In [ ]:
# 1. fit() 메서드 구현하기
'''
fit() 메서드의 기본 구조는 3장의 Neuron 클래스와 같다. 단, 활성화 함수(Activation Function)이 추가된 점이 다르다.
- 역방향 계산에는 로지스틱 손실 함수의 도함수를 적용한다.
- 앞에서 초기화하지 않은 가중치는 np.ones() 함수를 이용하여 간단히 1로 초기화하고 절편은 0으로 초기화한다.
'''
def fit(self, x, y, epochs = 100):
    self.w = np.ones(x.shape[1])                            # 가중치를 초기화한다.
    self.b = 0                                              # 절편을 초기화한다.
    for i in range(epochs):                                 # epochs만큼 반복한다.
        for x_i, y_i in zip(x, y):                          # 모든 샘플에 대해 반복한다.
            z = self.forpass(x_i)                           # 정방향 계산
            a = self.activation(z)                          # 활성화 함수 적용
            err = -(y_i - a)                                # 오차 계산
            w_grad, b_grad = self.backprop(x_i, err)        # 역방향 계산
            self.w -= w_grad                                # 가중치 업데이트
            self.b -= b_grad                                # 절편 업데이트

In [ ]:
# 2. activation() 메서드 구현
# activation() 메서드에는 시그모이드 함수가 사용되어야 한다.
# 시그모이드 함수는 자연 상수의 지수 함수를 계산하는 넘파이의 np.exp() 함수를 사용하여, 간단히 만들 수 있다.
def activation(self, z):
    a = 1 / (1 + np.exp(-z)) # 시그모이드 계산
    return a

In [ ]:
'''
예측하는 메서드 구현
- 새로운 샘플에 대한 예측값을 계산할 때 forpass() 메서드를 사용했다.
    - 여러 개의 샘플을 한꺼번에 예측하려면, forapss() 메서드를 여러 번 호출하게 되는데 이 작업이 번거롭다.
    - 분류에서는 활성화 함수와 임계 함수도 적용해야 하므로, 새로운 샘플에 대한 예측값을 계산해 주는 메서드인 predict() 메서드를 만들어본다.
'''

In [ ]:
# 1. predict() 메서드 구현하기
'''
predict() 메서드의 매개변수 값으로 입력값 x가 2차원 배열로 전달된다고 가정하고 구현한다.
- 예측값은 입력값ㅇ르 선형 함수, 활성화 함수, 임계 함수 순서로 통과시키면 구할 수 있다.
- 앞에서 forpass()와 activation() 메서드를 이미 구현했으니, predict() 메서드는 다음과 같이 간단하게 구현할 수 있다.
'''
def predict(self, x):
    z = [self.forpass(x_i) for x_i in x]        # 선형 함수 적용
    a = self.activation(np.array(z))            # 활성화 함수 적용
    return a > 0.5                              # 계단 함수 적용

'''
z의 계산으로 파이썬의 리스트 내포(list comprehension) 문법을 사용했다.
- 리스트 내포란, 대괄호 안에 for문을 삽입하여 새 리스트를 만드는 간결한 문법이다.
- x의 행을 하나씩 꺼내어, forpass() 메서드에 적용하고 그 결과를 이용하여 새 리스트 z로 만드는 것이다.
    - 즉, z는 곧바로 넘파이 배열로 바뀌어 activation() 메서드에 전달한다.
'''

In [ ]:
'''
로지스틱 회귀 모델 훈련시키기
- 준비한 데이터 셋을 사용하여, 로지스틱 회귀 모델을 훈련해 보고 정확도도 측정해 본다.
'''

In [ ]:
# 1. 모델 훈련하기
'''
모델을 훈련하는 방법은 3장과 동일하다
LogisticNeuron 클래스의 객체를 만든 후, 훈련 세트와 함께 fit() 메서드를 호출하면 된다.
'''
neuron = LogisticNeuron()
neuron.fit(X_train, y_train)

In [ ]:
# 2. 테스트 세트 사용하여 모델의 정확도 평가하기
# 위 코드를 통해, 훈련이 끝난 모델에 테스트 세트를 사용하여 예측값을 넣고 예측한 값이 맞는지 비교한다.
np.mean(neuron.predict(X_test) == y_test)
'''
predict() 메서드의 반환값은 True나 False로 채워진 (m, )크기의 배열이고 y_test는 0또는 1로 채워진 (m, ) 크기의 배열이므로 바로 비교할 수 있다.
np.mean() 함수는 매개변수 값으로 전달한 비교문 결과(넘파이 배열)의 평균을 계산한다.
- 즉, 계산 결과 0.82... 는 올바르게 예측한 샘플의 비율이 된다.
    - 이를 정확도(accuracy)라고도 한다.
    - 즉, 82%의 정확도가 나온것이다.
'''

In [ ]:
# 손실 함수의 결과값 조정해서 저장 기능 추가하기

def __init__(self):
    self.w = None
    self.b = None
    self.lossed = []

def fit(self, x, y, epochs=100):

    for i in index:                                         # 모든 샘플에 대해 반복한다.
        z = self.forpass(x[i])                              # 정방향 계산
        a = self.activation(z)                              # 활성화 함수 적용
        err  = -(y[i] - a)                                  # 오차 계산
        w_grad, b_grad = self.backprop(x[i], err)           # 역방향 계산
        self.w -= w_grad                                    # 가중치 업데이트
        self.b -= b_grad                                    # 절편 업데이트
        # 안전한 로그 계산을 위해 클리핑한 후 손실을 누적한다.
        a = np.clip(a, 1e-10, 1-1e-10)                      
        loss += -(y[i]*np.log(a) + (1-y[i])*np.log(1-a))    # 에포크바다 평균 손실을 저장한다.
    
    self.lossed.append(loss/len(y))

In [ ]:
# 매 에포크마다 훈련 세트의 샘플 순서 섞어서 사용하기
def fit(self, x, y, epochs = 100):
    self.w = np.ones(x.shape[1])                                     # 가중치 초기화
    self.b = 0                                                       # 절편 초기화
    for i in range(epochs):                                             
        loss = 0
        indexes = np.random.permutation(np.arange(len(x)))           # 인덱스 섞기
        for i in indexes:                                            # 모든 샘플에 대하여 반복
            z = self.forpass(x[i])                                   # 정방향 계산
            a = self.activation(z)                                   # 활성화 함수 적용
            err = -(y[i] - a)                                        # 오차 계산
            w_grad, b_grad = self.backprop(x[i], err)                # 역방향 계산
            self.w -= w_grad                                         # 가중치 업데이트
            self.b -= b_grad                                         # 절편 업데이트
            a = np.clip(a, 1e-10, 1-1e-10)                           # 안전한 로그 계산을 위하여 클리핑한 후 손실 누적
            loss += -(y[i]*np.log(a) + (1-y[i])*np.log(1-a))        # 에포크마다 평균 손실 저장
        self.losses.append(loss/len(y))

In [ ]:
# score() 메서드 추가하기
def predict(self, x):
    z = [self.forpass(x_i) for x_i in x] # 정방향 계산
    return np.array(z) > 0               # 게단 함수 적용

def score(self, x, y):
    return np.mean(self.predict(x) == y)

'''
시그모이드 함수의 출력값은 0 ~ 1 사이의 확률값이고, 양성 클래스를 판단하는 기준은 0.5이상이다.
그런데 z가 0보다 크면 시그모이드 함수의 출력값은 0.5보다 크고 z가 0보다 작으면 시그모이드 함수의 출력값은 0.5보다 작다.
그래서 predict() 메서드에는 굳이 시그모이드 함수를 사용하지 않아도 된다.
- z가 0보다 큰지, 작은지만 따지면 되기 때문이다.
그래서 predict() 메서드에는 로지스틱 함수를 적용하지 않고, z값의 크기만 비교하여 결과를 반환하면 된다.
'''

In [ ]:
# 전체 코드
class SingleLayer:
    def __init__(self):
        self.w = None
        self.b = None
        self.losses = []

    def forpass(self, x):
        z = np.sum(x * self.w) + self.b                         # 직선 방정식 계산
        return z
    
    def backprop(self, x, err):
        w_grad = x * err                                        # 가중치에 대한 그레이디언트 계산
        b_grad = 1 * err                                        # 절편에 대한 그레이디언트 계산
        return w_grad, b_grad
    
    def add_bias(self, x):
        return np.c_[np.ones((x.shape[0], 1)), x]               # 행렬의 맨 앞에 1로 채워진 열 벡터를 추가한다.
    
    def activation(self, z):
        a = 1 / (1 + np.exp(-z))                                # 시그모이드 계산
        return a

    def fit(self, x, y, epochs = 100):
        self.w = np.ones(x.shape[1])                                     # 가중치 초기화
        self.b = 0                                                       # 절편 초기화
        for i in range(epochs):                                             
            loss = 0
            indexes = np.random.permutation(np.arange(len(x)))           # 인덱스 섞기
            for i in indexes:                                            # 모든 샘플에 대하여 반복
                z = self.forpass(x[i])                                   # 정방향 계산
                a = self.activation(z)                                   # 활성화 함수 적용
                err = -(y[i] - a)                                        # 오차 계산
                w_grad, b_grad = self.backprop(x[i], err)                # 역방향 계산
                self.w -= w_grad                                         # 가중치 업데이트
                self.b -= b_grad                                         # 절편 업데이트
                a = np.clip(a, 1e-10, 1-1e-10)                           # 안전한 로그 계산을 위하여 클리핑한 후 손실 누적
                loss += -(y[i]*np.log(a) + (1-y[i])*np.log(1-a))        # 에포크마다 평균 손실 저장
            self.losses.append(loss/len(y))

    def predict(self, x):
        z = [self.forpass(x_i) for x_i in x] # 정방향 계산
        return np.array(z) > 0               # 게단 함수 적용

    def score(self, x, y):
        return np.mean(self.predict(x) == y)

In [ ]:
# 단일층 신경망 훈련하기
# 1. 단일층 신경망 훈련하고 정확도 출력하기
'''
SingleLayer 객체를 만들고 훈련 세트(X_train, y_train)로 이 신경망을 훈련한 다음, score() 메서드로 정확도를 출력한다.
'''
layer = SingleLayer()
layer.fit(X_train, y_train)
layer.score(X_test, y_test)

'''
정확도는 0.9298245614035088로 이전보다 좋아졌다.
LogisticNeuron과 마찬가지로 fit() 메서드의 에포크(epochs) 매개변수의 기본값 100을 그대로 사용했는데도 이렇게 성능이 놓아진
이유는 에포크마다 훈련 세트를 무작위로 섞어 손실 함수의 값을 줄였기 때문이다.
'''

In [ ]:
# 2. 손실 함수 누적값 확인하기
plt.plot(layer.losses)
plt.xlabel("epoch")
plt.ylabel("loss")
plt.show()
'''
그래프를 확인하보니 로지스틱 손실 함수의 값이 에포크가 진행됨에 따라 감소하고 있음을 확인할 수 있다.
아직까지는 은닉층을 사용하지 않았기에, 이 단일층 신경망은 로지스틱 회귀나 퍼셉트론 알고리즘과 매우 비슷하다.
'''

In [ ]:
# 사이킷런의 경사 하강법이 구현된 클래스는 SGDClassifier이다.
# 이 클래스는 로지스틱 회귀 문제 외에도 여러 가지 문제에 경사 하강법을 적용할 수 있다.
# SGDClassifier 클래스를 통하여 로지스틱 회귀 문제를 간단히 해결 한다.

In [ ]:
# 사이킷런으로 경사 하강법 적용하기
# 1. 로지스틱 손실 함수 지정하기
# SGDClassifier 클래스에 로지스틱 회귀를 적용하려면 loss 매개변수에 손실 함수로 log를 지정한다.
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(loss='log', max_iter = 100, tol = 1e-3, random_state = 42)
'''
- max_iter를 통해 반복 횟수를 100으로 지정하고, 반복 실행했을 때 결과를 동일하게 재현하기 위해 random_state를 통해 난수 초깃값을 42로 설정한다.
- 반복할 때마다 로지스틱 손실 함수의 값이 tol에 지정한 값만큼 감소되지 않으면, 반복을 중단하도록 설정한다.
- 만일 tol의 값을 설정하지 않으면 max_iter의 값을 늘리라는 경고가 방생한다.
    - 이는 모델의 로지스틱 손실 함수의 값이 최적값으로 수렴할 정도로 충분한 반복 횟수를 입력했는지 사용자에게 알려주므로 유용하다.
'''

In [ ]:
# 2. 사이킷런으로 훈련하고 평가하기
'''
사이킷런의 SGDClassifier 클래스에는 지금까지 우리가 직접 구현한 메서드가 이미 준비되어 있다.
- 사이킷런의 fit() 메서드로 훈련하고 score() 메서드로 정확도를 계산하면 된다.
'''
# sgd.fit(X_train, y_train)
sgd.score(X_test, y_test)


In [ ]:
# 3. 사이킷런으로 예측하기
'''
사이킷런은 입력 데이터로 2차원 배열만 받아들인다.
즉, 샘플 하나를 주입하더라도 2차원 배열로 만들어야 한다.
아래에서는 배열의 슬라이싱을 사용하여 테스트 세트에서 10개의 샘플만 뽑아 예측한다.
'''
sgd.predict(X_test[0:10])